<a href="https://colab.research.google.com/github/raynardj/python4ml/blob/master/experiments/Sentiment_Single_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
# For 1st timer, load the google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np

In [17]:
# Install ray's tool box
!mkdir -p $HOME/data
!!pip install git+https://github.com/raynardj/forge

['Collecting git+https://github.com/raynardj/forge',
 '  Cloning https://github.com/raynardj/forge to /tmp/pip-req-build-8bcyh_hm',
 'Requirement already satisfied (use --upgrade to upgrade): ai-forge==0.1.0 from git+https://github.com/raynardj/forge in /usr/local/lib/python3.6/dist-packages',
 'Requirement already satisfied: flask==0.12.4 in /usr/local/lib/python3.6/dist-packages (from ai-forge==0.1.0) (0.12.4)',
 'Requirement already satisfied: flask_appbuilder==1.10.0 in /usr/local/lib/python3.6/dist-packages (from ai-forge==0.1.0) (1.10.0)',
 'Requirement already satisfied: pandas>=0.18.0 in /usr/local/lib/python3.6/dist-packages (from ai-forge==0.1.0) (0.24.2)',
 'Requirement already satisfied: tqdm>=4.25.0 in /usr/local/lib/python3.6/dist-packages (from ai-forge==0.1.0) (4.31.1)',
 'Requirement already satisfied: itsdangerous>=0.21 in /usr/local/lib/python3.6/dist-packages (from flask==0.12.4->ai-forge==0.1.0) (1.1.0)',
 'Requirement already satisfied: Jinja2>=2.4 in /usr/local/l

In [25]:
train = pd.read_csv(DATA/'quora_train.csv')

split_ = (np.random.rand(len(train))>0.7)
train_df = train.loc[~split_].reset_index().drop("index",axis=1)
valid_df = train.loc[split_].reset_index().drop("index",axis=1)

from forgebox.ftorch.prepro import DF_Dataset,Arr_Dataset,Seq_Dataset, fuse
from torch.utils.data.dataloader import DataLoader

# Text transformed to sequence data
train_txt_ds = Seq_Dataset("train",train_df.question_text,seq_len=SEQ_LEN,vocab_path=DATA/"vocab.json",bs= BS ,build_vocab=True,sep_tok = " ", fixlen=True)
valid_txt_ds = Seq_Dataset("valid",valid_df.question_text,seq_len=SEQ_LEN,vocab_path=DATA/"vocab.json",bs= BS ,build_vocab=False,sep_tok = " ", fixlen=True)

# Label transformed to 1/0 array
train_lbl_ds = Arr_Dataset(train_df.target.values,bs=BS)
valid_lbl_ds = Arr_Dataset(valid_df.target.values,bs=BS)

# Combine 2 datasets
train_ds = fuse(train_txt_ds,train_lbl_ds)
valid_ds = fuse(valid_txt_ds,valid_lbl_ds)

# Testing data with dataloader
dl = DataLoader(train_ds,batch_size=1,shuffle=True)
testgen = iter(dl)
seq_, label_ = next(testgen)

seq_, label_ 

train sequence type: <class 'pandas.core.series.Series'>
train sequence total_length type: 913544
valid sequence type: <class 'pandas.core.series.Series'>
valid sequence total_length type: 392578


(tensor([[[  20,   19, 1117,  ...,    0,    0,    0],
          [ 154,   16, 1374,  ...,    0,    0,    0],
          [  11,   14,   10,  ...,    0,    0,    0],
          ...,
          [  11,   78,  176,  ...,    0,    0,    0],
          [   4,   13,   36,  ...,    0,    0,    0],
          [  11,   60,    3,  ...,    0,    0,    0]]]),
 [tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]])])

In [0]:
from forgebox.ftorch import layers
import forge

In [22]:
from pathlib import Path
DATA = Path('/content/gdrive/My Drive/data/quora/')

import torch
DIM = 300
CUDA = torch.cuda.is_available()
BS = 64
MODEL_NAME = "lstm"
TRY_NAME = "smaller_hidden"
SEQ_LEN = 30

print("Is gpu available",CUDA)

Is gpu available False


In [0]:
from forgebox.ftorch.layers import LayerNorm,GELU,Attention, MultiHeadedAttention,TransformerBlock
import torch
from torch import nn

class LM_RNN_ATTN(nn.Module):
    def __init__(self,vocab_size,
                 hs = DIM):
        super().__init__()

        self.vocab_size = vocab_size
        self.emb = nn.Embedding(self.vocab_size,hs)
        self.attention = MultiHeadedAttention(4,hs)
        self.rnn = nn.LSTM(input_size = hs, hidden_size = hs,num_layers = 1,batch_first = True)
        
        self.output = nn.Sequential(*[
            nn.Linear(hs*2,512), 
            nn.ReLU(),
            nn.Linear(512,1),
            nn.Sigmoid(),
                                     ])
        
    def forward(self,x):
        x = self.emb(x)
        x1 = self.attention(x,x,self.rnn(x)[0])
        x2 = self.attention(x,x,self.rnn(x.flip(1))[0])
        x = torch.cat([x1[:,-1,:],x2[:,-1,:]], dim = -1)

        x = self.output(x)
        return x

In [0]:
md = LM_RNN_ATTN(train_txt_ds.vocab_size, DIM)

In [0]:
from forgebox.ftorch.utils import load_model

In [0]:
load_model(md,DATA/"attn_rnn_cpu.npy")